In [1]:
from termcolor import colored
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
# other python utilities 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity
# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)


In [2]:
df = pd.read_csv("D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Oct_Forth_projectType.csv")

categories_cols = []

print(colored('categorizing the following columns for less memory usage...', 'green'))
for col in df.columns:
    #print(str(col))
    cats = len(df[col].unique())
    #print(cats)
    if cats<=10:
        categories_cols.append(col)
        print(col)
        
df[categories_cols] = df[categories_cols].astype('category')

categorizing the following columns for less memory usage...
Project Grade Level Category
Project Resource Category
Project Current Status
School Metro Type
School State
Donor State


In [ ]:
donors_projects_df = donor_project.pivot('Donor ID', 'Project ID', 'Donation Amount').fillna(0)

donors_projects = donors_projects_df.values
donors_id = donors_projects_df.index
projects_id = donors_projects_df.columns

print(f'length of donors: {len(donors_id)}')
print(f'length of projects: {len(projects_id)}')

In [ ]:
U, sigma, Vt = svds(donors_projects, k=50)
sigma = np.diag(sigma)
all_donor_predicted_preference = np.dot(np.dot(U, sigma), Vt)

svg_predicted_preference_df = pd.DataFrame(all_donor_predicted_preference,
                          columns=projects_id,
                          index=donors_id).transpose()

svg_predicted_preference_df.shape # shape[0] == len(donor_ids)

In [ ]:
class CFRecommender:
    

    """
     We only need to get the ids of donors and sort the projects they might 
    values in front of each that have been calculated using svd and multiplication
    of the donor-donation matrix and the eigen vectors
     """

    MODEL_NAME = 'Collaborative Filtering'


    def __init__(self, svg_predicted_preference_df, projects = None):
        self.svg_predicted_preference_df = svg_predicted_preference_df
        self.df = df

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_projects(self, donor_id, projects_to_ignore=[], top_n=10, df_test = df_test):
        
        
        """This function gets a list of projects have to be ignored
        and outputs a dataframe of all the projects this person may 
        like
        
        
        """
        donated_projects_test = df_test_donor_index.loc[donor_id, 'Project ID']
        if type(donated_projects_test) == pd.Series:
            donated_projects_test = set(donated_projects_test)
        else: donated_projects_test = set([donated_projects_test])

        # projects that the user has donated to in both testing and training set
        wanted_in_test = []
        for project_id in projects_to_ignore:
            if project_id in donated_projects_test:
                wanted_in_test.append(project_id)

        if wanted_in_test != []: projects_to_ignore = set(projects_to_ignore).difference(wanted_in_test)

        donor_predictions = self.svg_predicted_preference_df[donor_id].reset_index().rename(columns={donor_id: 'recommStrength'})
        
        # making sure we have all projects we want and sorting them
        recomm_df = donor_predictions[~donor_predictions['Project ID'].isin(projects_to_ignore)].sort_values(by='recommStrength', ascending=False).head(top_n)

        projects_desctiption = self.df[['Project ID', 'Project Title', 'Project Need Statement','Project Resource Category', 'Project Subject Category Tree']]
        projects_desctiption = projects_desctiption.drop_duplicates(keep='first')
        # merging with projects dataset to get the descriptions
        recomm_df_merged = pd.merge(left = recomm_df, right = projects_desctiption, how='left', on='Project ID')#[['recommStrength', 'Project ID', 'Project Title', 'Project Need Statement']]
        

        return recomm_df_merged
